title: Testing a new UI in the product selection flow
author: Fabio Schmidt-Fischbach  
date: 2020-07-09  
region: EU  
summary: We recently started testing a new UI in the product selection flow. This deep dive focuses less on the UX component but more on the directly business relevant metrics in both the short and long run. Does the new UX lead to more premium customers? Do more or less activate? The new UX performs on various dimensions slightly worse than the control on short-medium term metrics like KYC conversion and first time activation. The main positive result is that the drop in premium (mainly, Metal) observable at KYC shrinks once we look at activation suggesting that fewer Metal users pick the product but never activate.
tags: product selection, onboarding, funnel, acquire, memberships, premium

In [1]:
import pandas as pd
import os
import seaborn as sns
from statsmodels.stats.proportion import proportions_ztest
import altair as alt

### Summary 

1. Sample size: 80k users per variant. 
2. % product selection to KYCc conversion is 0.4pp lower in variant.
3. % premium x-sell at KYC is 0.75pp lower in variant. On a positive note, % xsell to Business Standard is up by 1.4pp.  
4. % product selection to ftMAU is 0.6pp lower in the variant. 
5. % product selection to premium ftMAU is 0.3pp lower in variant. 

The new UX performs on various dimensions slightly worse than the control on short-medium term metrics like KYC conversion and first time activation.    

The main positive result is that the drop in premium (mainly, Metal) observable at KYC shrinks once we look at activation suggesting that fewer Metal users pick the product but never activate.   



### Setup 

We recently started testing a new UI in the product selection flow. You can explore how it looks like and how users so far navigate it in this deep dive [http://research.tech26.de/reports/product_selection_new_ui_report.html].    

This deep dive focuses less on the UX component but more on the directly business relevant metrics in both the short and long run. Does the new UX lead to more premium customers? Do more or less activate? 

In [2]:
query = """

select fe.*, dbt.zrh_users.user_id, dbt.zrh_users.kyc_first_initiated
from dbt.zrh_solutions_fe as fe  
left join dbt.zrh_users using (user_created) 
where se_property in ('product_selection_new_ui', 'product_selection_new_ui_control')

"""

### Sample size 

In [5]:
df = pd.read_csv("product_selection.csv")

df = df.groupby(["se_property"])["user_id"].agg("nunique").reset_index()

alt.Chart(df).mark_bar().encode(
    x=alt.X("se_property:N", axis=alt.Axis(title="Group")),
    y=alt.Y("user_id:Q", axis=alt.Axis(title="Number of users")),
).properties(width=500, height=500, title="Sample size")

/Users/fabioschmidt-fischbach/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


alt.Chart(...)

In [7]:
df = pd.read_csv("product_selection.csv")

df = df.groupby(["se_property", "created"])["user_id"].agg("nunique").reset_index()
df["created"] = df["created"].astype(str)

alt.Chart(df).mark_line().encode(
    x=alt.X("created:N", axis=alt.Axis(title="Group")),
    y=alt.Y("user_id:Q", axis=alt.Axis(title="Number of users")),
    color="se_property",
).properties(width=500, height=500, title="Users over time")

alt.Chart(...)

### 1. Does the new UX lead to more or less KYCc? 

We see a slightly lower KYCc conversion in the treatment group. 

In [16]:
df = pd.read_csv("product_selection.csv")

# drop users who joined in the last 7 days
df["age"] = (pd.to_datetime("today") - pd.to_datetime(df["user_created"])).dt.days
df = df.loc[df["age"] >= 7, :]

# look at who managed to pass kycc within 7 days
df["kycc"] = 0
df.loc[
    (
        pd.to_datetime(df["kyc_first_completed"]) - pd.to_datetime(df["user_created"])
    ).dt.days
    <= 7,
    "kycc",
] = 1

df = df.groupby(["se_property", "user_id"])["kycc"].agg("max").reset_index()
df = df.groupby(["se_property"])["kycc"].agg("mean").reset_index()

alt.Chart(df).mark_bar().encode(
    x=alt.X("se_property:N", axis=alt.Axis(title="Group")),
    y=alt.Y("kycc:Q", axis=alt.Axis(title="% of KYCc within 7 days", format="%")),
    color="se_property",
).properties(width=500, height=500, title="% of KYCc within 7 days")

alt.Chart(...)

In [17]:
df.head()

,se_property,kycc
0,product_selection_new_ui,0.545873
1,product_selection_new_ui_control,0.551813


In [18]:
df = pd.read_csv("product_selection.csv")

# drop users who joined in the last 7 days
df["age"] = (pd.to_datetime("today") - pd.to_datetime(df["user_created"])).dt.days
df = df.loc[df["age"] >= 7, :]

# look at who managed to pass kycc within 7 days
df["kycc"] = 0
df.loc[
    (
        pd.to_datetime(df["kyc_first_completed"]) - pd.to_datetime(df["user_created"])
    ).dt.days
    <= 7,
    "kycc",
] = 1

df = df.groupby(["se_property", "user_id"])["kycc"].agg("max").reset_index()

data = df.groupby("se_property")["kycc"].agg(["count", "sum"]).reset_index()

# run z test. (two sided)
stat, pval = proportions_ztest(data["sum"], data["count"], alternative="smaller")

print(
    "The z-score for this test is %s which corresponds to a p-value of %s"
    % (round(stat, 2), round(pval, 4))
)

if pval < 0.05:
    print("The difference is significant.")
else:
    print("The difference is not signficiant.")

/Users/fabioschmidt-fischbach/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


The z-score for this test is -2.41 which corresponds to a p-value of 0.0079
The difference is significant.


#### 2. Does the new UX lead to more or less premium users at KYC? 

KYC X-sell to premium goes down by 0.7pp - the difference is significant. 

In [19]:
df = pd.read_csv("product_selection.csv")

# drop users who joined in the last 7 days
df["age"] = (pd.to_datetime("today") - pd.to_datetime(df["user_created"])).dt.days
df = df.loc[df["age"] >= 7, :]

# only consider kycc
df = df.loc[df["kyc_first_completed"].isna() == False, :]

df["product_group"] = "Non-premium"
df.loc[
    df["product_id"].isin(
        ["BLACK_CARD_MONTHLY", "METAL_CARD_MONTHLY", "BUSINESS_BLACK", "BUSINESS_METAL"]
    ),
    "product_group",
] = "Premium"

df = (
    df.groupby(["se_property", "product_group"])["user_id"].agg("nunique").reset_index()
)
df["perc"] = (
    100 * df["user_id"] / df.groupby(["se_property"])["user_id"].transform("sum")
)

alt.Chart(df.loc[df["product_group"] == "Premium", :]).mark_bar().encode(
    x=alt.X("se_property:N", axis=alt.Axis(title="Group")),
    y=alt.Y("perc:Q", axis=alt.Axis(title="KYC x-sell %")),
).properties(width=400, height=400, title="Premium KYC-xsell")

alt.Chart(...)

In [20]:
df.head()

,se_property,product_group,user_id,perc
0,product_selection_new_ui,Non-premium,42042,89.630324
1,product_selection_new_ui,Premium,4864,10.369676
2,product_selection_new_ui_control,Non-premium,42074,88.203602
3,product_selection_new_ui_control,Premium,5627,11.796398


In [21]:
df = pd.read_csv("product_selection.csv")

# drop users who joined in the last 30 days
df["age"] = (pd.to_datetime("today") - pd.to_datetime(df["user_created"])).dt.days
df = df.loc[df["age"] >= 7, :]

# only consider kycc
df = df.loc[df["kyc_first_completed"].isna() == False, :]

df["product_group"] = 0
df.loc[
    df["product_id"].isin(
        ["BLACK_CARD_MONTHLY", "METAL_CARD_MONTHLY", "BUSINESS_BLACK", "BUSINESS_METAL"]
    ),
    "product_group",
] = 1

df = df.groupby(["se_property", "user_id"])["product_group"].agg("max").reset_index()
data = df.groupby("se_property")["product_group"].agg(["count", "sum"]).reset_index()

# run z test. (two sided)
stat, pval = proportions_ztest(data["sum"], data["count"], alternative="smaller")

print(
    "The z-score for this test is %s which corresponds to a p-value of %s"
    % (round(stat, 2), round(pval, 4))
)

if pval < 0.05:
    print("The difference is significant.")
else:
    print("The difference is not signficiant.")

/Users/fabioschmidt-fischbach/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


The z-score for this test is -6.99 which corresponds to a p-value of 0.0
The difference is significant.


Which products are driving this drop in premium xsell? Mainly metal. We also see a surge in Business-Standard.

In [22]:
df = pd.read_csv("product_selection.csv")

# drop users who joined in the last 7 days
df["age"] = (pd.to_datetime("today") - pd.to_datetime(df["user_created"])).dt.days
df = df.loc[df["age"] >= 7, :]

# only consider kycc
df = df.loc[df["kyc_first_completed"].isna() == False, :]

df = df.groupby(["se_property", "product_id"])["user_id"].agg("nunique").reset_index()
df["perc"] = (
    100 * df["user_id"] / df.groupby(["se_property"])["user_id"].transform("sum")
)

alt.Chart(df).mark_bar().encode(
    x=alt.X("se_property:N", axis=alt.Axis(title="Group")),
    y=alt.Y("perc:Q", axis=alt.Axis(title="KYC x-sell %")),
    color="se_property:N",
).properties(width=200, height=200, title="KYC-xsell").facet(
    facet="product_id", columns=3
).resolve_scale(
    y="independent"
)

alt.FacetChart(...)

In [23]:
df.head(100)

,se_property,product_id,user_id,perc
0,product_selection_new_ui,BLACK_CARD_MONTHLY,1563,3.332196
1,product_selection_new_ui,BUSINESS_BLACK,916,1.952842
2,product_selection_new_ui,BUSINESS_CARD,6953,14.823264
3,product_selection_new_ui,BUSINESS_METAL,501,1.068094
4,product_selection_new_ui,FLEX_ACCOUNT_MONTHLY,1779,3.792692
5,product_selection_new_ui,METAL_CARD_MONTHLY,1884,4.016544
6,product_selection_new_ui,STANDARD,33310,71.014369
7,product_selection_new_ui_control,BLACK_CARD_MONTHLY,1634,3.425505
8,product_selection_new_ui_control,BUSINESS_BLACK,980,2.054464
9,product_selection_new_ui_control,BUSINESS_CARD,6367,13.347729


#### 3. Does the new UX lead to more or less active users? 
In the following we drop all users that joined in the last 30 days - as we consider them to be too young to activate. 

It is very common to look at KYC numbers, but KYC customers do not necessarily make us money if they are not active. As a next step we want to investigate whether the new UX had an impact on activation. Do more users activate? 

I categorize all our participants in either a) activated user or b) non activated user and split the activated users by premium or non-premium.   

This allows us to answer the following question:
- For each user we push into product selection, what % never activates, % activates but without premium and activates but with premium? 
- the new UX features a 0.3pp lower conversion % of users converting to paying premium customers than the control. 
- the new UX features a 0.6pp lower conversion to any activation. 



In [24]:
df = pd.read_csv("product_selection.csv")

# drop users who joined in the last 30 days
df["age"] = (pd.to_datetime("today") - pd.to_datetime(df["user_created"])).dt.days
df = df.loc[df["age"] >= 30, :]

df = df.groupby(["se_property", "user_id"])["ever_top_up"].agg("max").reset_index()
df = df.groupby(["se_property"])["ever_top_up"].agg("mean").reset_index()

chart = (
    alt.Chart(df)
    .mark_bar()
    .encode(
        x=alt.X("se_property:N", axis=alt.Axis(title="Group")),
        y=alt.Y(
            "ever_top_up:Q",
            axis=alt.Axis(title="% ftMAU out of all participants", format="%"),
        ),
        color="se_property:N",
    )
    .properties(width=400, height=400, title="% ftMAU out of all users")
)

/Users/fabioschmidt-fischbach/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [25]:
# Are there some time-trends to be wary off? Not v surprisingly,
# older cohorts have higher chances to have ever turned MAU.
# This aside it generally looks like the two groups are very close / control has a slight edge over treatment.

df = pd.read_csv("product_selection.csv")

# drop users who kycc in the last 30 days
df["age"] = (
    pd.to_datetime("today") - pd.to_datetime(df["kyc_first_completed"])
).dt.days
df = df.loc[df["age"] >= 30, :]

df["kyc_first_completed"] = pd.to_datetime(df["kyc_first_completed"]).dt.date.astype(
    str
)

df = (
    df.groupby(["se_property", "kyc_first_completed", "user_id"])["ever_top_up"]
    .agg("max")
    .reset_index()
)
df = (
    df.groupby(["se_property", "kyc_first_completed"])["ever_top_up"]
    .agg("mean")
    .reset_index()
)

chart = (
    alt.Chart(df.loc[df["kyc_first_completed"] >= "2020-06-01"])
    .mark_line()
    .encode(
        x=alt.X("kyc_first_completed:N", axis=alt.Axis(title="Group")),
        y=alt.Y(
            "ever_top_up:Q",
            axis=alt.Axis(title="% ftMAU out of all participants", format="%"),
        ),
        color="se_property:N",
    )
    .properties(width=400, height=400, title="% ftMAU out of all users")
)

In [26]:
# Now, let's look at % ftMAU conditional on passing KYC.
# We again see that the control group maintains a slight slight edge of 1pp.
df = pd.read_csv("product_selection.csv")

# drop users who joined in the last 30 days
df["age"] = (
    pd.to_datetime("today") - pd.to_datetime(df["kyc_first_completed"])
).dt.days
df = df.loc[df["age"] >= 30, :]

df = df.loc[df["kyc_first_completed"].isna() == False, :]

df = df.groupby(["se_property", "user_id"])["ever_top_up"].agg("max").reset_index()
df = df.groupby(["se_property"])["ever_top_up"].agg("mean").reset_index()

chart = (
    alt.Chart(df)
    .mark_bar()
    .encode(
        x=alt.X("se_property:N", axis=alt.Axis(title="Group")),
        y=alt.Y(
            "ever_top_up:Q", axis=alt.Axis(title="% ftMAU out of all KYCc", format="%")
        ),
        color="se_property:N",
    )
    .properties(width=400, height=400, title="% ftMAU out of all KYCc")
)

In [34]:
df = pd.read_csv("product_selection.csv")

# drop users who joined in the last 30 days
df["age"] = (pd.to_datetime("today") - pd.to_datetime(df["user_created"])).dt.days
df = df.loc[df["age"] >= 30, :]

df["product"] = "No product chosen"
df.loc[
    df["product_id"].isin(
        ["BLACK_CARD_MONTHLY", "METAL_CARD_MONTHLY", "BUSINESS_METAL", "BUSINESS_BLACK"]
    )
    == True,
    "product",
] = "Premium ftMAU"
df.loc[
    df["product_id"].isin(
        ["BLACK_CARD_MONTHLY", "METAL_CARD_MONTHLY", "BUSINESS_METAL", "BUSINESS_BLACK"]
    )
    == False,
    "product",
] = "Non-premium ftMAU"

df = (
    df.groupby(["se_property", "product", "user_id"])["ever_top_up"]
    .agg("max")
    .reset_index()
)
df.loc[df["ever_top_up"] == 0, "product"] = "Never top up"

df = df.groupby(["se_property", "product"])["user_id"].agg("nunique").reset_index()

df["perc"] = df["user_id"] / df.groupby(["se_property"])["user_id"].transform("sum")

alt.Chart(df).mark_bar().encode(
    x=alt.X("se_property:N", axis=alt.Axis(title="Group")),
    y=alt.Y(
        "perc:Q",
        axis=alt.Axis(title="% of users in experiment that turned MAU", format="%"),
    ),
    color="se_property:N",
).properties(
    width=200, height=200, title="% of users in experiment that turned MAU"
).facet(
    facet="product", columns=3
).resolve_scale(
    y="independent"
)

alt.FacetChart(...)

In [35]:
df.head(20)

,se_property,product,user_id,perc
0,product_selection_new_ui,Never top up,20733,0.608166
1,product_selection_new_ui,Non-premium ftMAU,11750,0.344666
2,product_selection_new_ui,Premium ftMAU,1608,0.047168
3,product_selection_new_ui_control,Never top up,20760,0.602053
4,product_selection_new_ui_control,Non-premium ftMAU,11977,0.347341
5,product_selection_new_ui_control,Premium ftMAU,1745,0.050606


Is the difference in % users in experiment that ever turn premium ftMAU significant? 

In [36]:
df = pd.read_csv("product_selection.csv")

# drop users who joined in the last 30 days
df["age"] = (pd.to_datetime("today") - pd.to_datetime(df["user_created"])).dt.days
df = df.loc[df["age"] >= 30, :]

df["product"] = "No product chosen"
df.loc[
    df["product_id"].isin(
        ["BLACK_CARD_MONTHLY", "METAL_CARD_MONTHLY", "BUSINESS_METAL", "BUSINESS_BLACK"]
    )
    == True,
    "product",
] = "Premium"
df.loc[
    df["product_id"].isin(
        ["BLACK_CARD_MONTHLY", "METAL_CARD_MONTHLY", "BUSINESS_METAL", "BUSINESS_BLACK"]
    )
    == False,
    "product",
] = "Non-premium"

df = (
    df.groupby(["se_property", "product", "user_id"])["ever_top_up"]
    .agg("max")
    .reset_index()
)

df["converted"] = 0
df.loc[(df["product"] == "Premium") & (df["ever_top_up"] == 1), "converted"] = 1

data = df.groupby("se_property")["converted"].agg(["count", "sum"]).reset_index()

# run z test. (two sided)
stat, pval = proportions_ztest(data["sum"], data["count"], alternative="smaller")

print(
    "The z-score for this test is %s which corresponds to a p-value of %s"
    % (round(stat, 2), round(pval, 4))
)

if pval < 0.05:
    print("The difference is significant.")
else:
    print("The difference is not signficiant.")

/Users/fabioschmidt-fischbach/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


The z-score for this test is -2.09 which corresponds to a p-value of 0.0184
The difference is significant.


How does this look by product? The main drop is stemming from Metal which looses 0.2pp. 

In [30]:
df = pd.read_csv("product_selection.csv")

# drop users who joined in the last 30 days
df["age"] = (pd.to_datetime("today") - pd.to_datetime(df["user_created"])).dt.days
df = df.loc[df["age"] >= 30, :]

df.loc[df["product_id"].isna() == True, "product_id"] = "No product chosen"

df = (
    df.groupby(["se_property", "product_id", "user_id"])["ever_top_up"]
    .agg("max")
    .reset_index()
)
df.loc[df["ever_top_up"] == 0, "product_id"] = "Never top up"

df = df.groupby(["se_property", "product_id"])["user_id"].agg("nunique").reset_index()

df["perc"] = df["user_id"] / df.groupby(["se_property"])["user_id"].transform("sum")

alt.Chart(df).mark_bar().encode(
    x=alt.X("se_property:N", axis=alt.Axis(title="Group")),
    y=alt.Y(
        "perc:Q",
        axis=alt.Axis(title="% of users in experiment that turned MAU", format="%"),
    ),
    color="se_property:N",
).properties(
    width=200, height=200, title="% of users in experiment that turned MAU"
).facet(
    facet="product_id", columns=3
).resolve_scale(
    y="independent"
)

/Users/fabioschmidt-fischbach/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


alt.FacetChart(...)

In [31]:
df.head(20)

,se_property,product_id,user_id,perc
0,product_selection_new_ui,BLACK_CARD_MONTHLY,545,0.015992
1,product_selection_new_ui,BUSINESS_BLACK,335,0.009830
2,product_selection_new_ui,BUSINESS_CARD,1856,0.054460
3,product_selection_new_ui,BUSINESS_METAL,48,0.001408
4,product_selection_new_ui,FLEX_ACCOUNT_MONTHLY,709,0.020804
5,product_selection_new_ui,METAL_CARD_MONTHLY,680,0.019953
6,product_selection_new_ui,Never top up,20729,0.608245
7,product_selection_new_ui,STANDARD,9178,0.269308
8,product_selection_new_ui_control,BLACK_CARD_MONTHLY,557,0.016157
9,product_selection_new_ui_control,BUSINESS_BLACK,360,0.010442
